In [7]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\jaesc2\GitHub\skforecast


In [8]:
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [9]:
# df 1000 series
np.random.seed(1)
n_series = 100
n = 100
index = pd.date_range(start='01-01-2010', periods=n, freq='D')
columns = ['series_' + str(i) for i in range(n_series)]
data = np.random.normal(loc=0, scale=1, size=(n, n_series))
df = pd.DataFrame(data, index=index, columns=columns)

columns_exog = ['exog_' + str(i) for i in range(n_series)]
data_exog = np.random.normal(loc=0, scale=1, size=(n, n_series))
df_exog = pd.DataFrame(data_exog, index=index, columns=columns_exog)

In [10]:
transformer_series = {col: StandardScaler() for col in columns}
transformer_series['_unknown_level'] = StandardScaler()

forecaster = ForecasterAutoregMultiSeries(
                    regressor = LinearRegression(),
                    lags = 5,
                    transformer_series = StandardScaler(),
                )
forecaster

============================ 
ForecasterAutoregMultiSeries 
============================ 
Regressor: LinearRegression() 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series names (levels): None 
Series encoding: ordinal 
Series weights: None 
Exogenous included: False 
Exogenous names: None 
Transformer for series: StandardScaler() 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: 
    {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2024-10-14 16:30:47 
Last fit date: None 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

In [11]:
forecaster.__repr__

<bound method ForecasterAutoregMultiSeries.__repr__ of ============================ 
ForecasterAutoregMultiSeries 
Regressor: LinearRegression() 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series names (levels): None 
Series encoding: ordinal 
Series weights: None 
Exogenous included: False 
Exogenous names: None 
Transformer for series: StandardScaler() 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: 
    {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2024-10-14 16:30:47 
Last fit date: None 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None 
>

In [5]:
forecaster.fit(series=df, exog=df_exog)
forecaster

============================ 
ForecasterAutoregMultiSeries 
============================ 
Regressor: LinearRegression() 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series names (levels): 
    series_0, series_1, series_2, series_3, series_4, series_5, series_6, series_7,
    series_8, series_9, series_10, series_11, series_12, series_13, series_14,
    series_15, series_16, series_17, series_18, series_19, series_20, series_21,
    series_22, series_23, series_24, ..., series_75, series_76, series_77,
    series_78, series_79, series_80, series_81, series_82, series_83, series_84,
    series_85, series_86, series_87, series_88, series_89, series_90, series_91,
    series_92, series_93, series_94, series_95, series_96, series_97, series_98,
    series_99 
Series encoding: ordinal 
Series weights: None 
Exogenous included: True 
Exogenous names: 
    exog_0, exog_1, exog_2, exog_3, exog_4, exog_5, exog_6, exog_7, exog_8, exog_9,
    exog_10, exog_11, exog_12, exog_13, exog_14, exog_15, exog_16, exog_17,
    exog_18, exog_19, exog_20, exog_21, exog_22, exog_23, exog_24, ..., exog_75,
    exog_76, exog_77, exog_78, exog_79, exog_80, exog_81, exog_82, exog_83,
    exog_84, exog_85, exog_86, exog_87, exog_88, exog_89, exog_90, exog_91,
    exog_92, exog_93, exog_94, exog_95, exog_96, exog_97, exog_98, exog_99 
Transformer for series: StandardScaler() 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: 
    'series_0': ['2010-01-01', '2010-04-10'], 'series_1': ['2010-01-01',
    '2010-04-10'], 'series_2': ['2010-01-01', '2010-04-10'], 'series_3':
    ['2010-01-01', '2010-04-10'], 'series_4': ['2010-01-01', '2010-04-10'], ...,
    'series_95': ['2010-01-01', '2010-04-10'], 'series_96': ['2010-01-01',
    '2010-04-10'], 'series_97': ['2010-01-01', '2010-04-10'], 'series_98':
    ['2010-01-01', '2010-04-10'], 'series_99': ['2010-01-01', '2010-04-10'] 
Training index type: DatetimeIndex 
Training index frequency: D 
Regressor parameters: 
    {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2024-10-14 16:30:12 
Last fit date: 2024-10-14 16:30:24 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

In [6]:
forecaster.__repr__

<bound method ForecasterAutoregMultiSeries.__repr__ of ============================ 
ForecasterAutoregMultiSeries 
Regressor: LinearRegression() 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series names (levels): 
    series_0, series_1, series_2, series_3, series_4, series_5, series_6, series_7,
    series_8, series_9, series_10, series_11, series_12, series_13, series_14,
    series_15, series_16, series_17, series_18, series_19, series_20, series_21,
    series_22, series_23, series_24, ..., series_75, series_76, series_77,
    series_78, series_79, series_80, series_81, series_82, series_83, series_84,
    series_85, series_86, series_87, series_88, series_89, series_90, series_91,
    series_92, series_93, series_94, series_95, series_96, series_97, series_98,
    series_99 
Series encoding: ordinal 
Series weights: None 
Exogenous included: True 
Exogenous names: 
    exog_0, exog_1, exog_2, exog_3, exog_4, exog_5, exog_6, exog_7, exog_8, exog_9,
    exog_10, exog_1

In [64]:
def aaa(trab=None):
    b = 1
    return trab, b

In [65]:
x, b = aaa()
x

In [66]:
b

1

In [28]:
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate

lags={'l1': None, 'l2': None}
expected = {'l1': None, 'l2': np.array([1, 2, 3, 4, 5])}

forecaster = ForecasterAutoregMultiVariate(
    LinearRegression(), level='l1', lags=5, steps=2
)
forecaster.lags_names

['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [21]:
import numpy as np

for k in forecaster.lags:
    if lags[k] is None:
        assert forecaster.lags[k] is None
        assert forecaster.lags_names[k] is None
    else:
        np.testing.assert_array_almost_equal(forecaster.lags[k], expected[k])
        assert forecaster.lags_names[k] == [f'lag_{i}' for i in range(1, lags[k] + 1)]

In [30]:
cols_to_create_lags = {'col1': 5, 'col2': None}

cols_to_create_lags.get('col3', None)

In [42]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10)), 
                        'l2': pd.Series(np.arange(100, 110))})
exog = None

forecaster = ForecasterAutoregMultiVariate(LinearRegression(), level='l1',
                                            lags={'l1': 3, 'l2': 3}, 
                                            steps=2, transformer_series=None)
results = forecaster._create_train_X_y(series=series, exog=exog)

{'l1': 'both', 'l2': 'X'}


In [45]:
np.array([[3., 4., 5., 6., 7., 8., 9.]])[1 - 1, :]

array([3., 4., 5., 6., 7., 8., 9.])

In [44]:
pd.Series(np.array([[3., 4., 5., 6., 7., 8., 9.]])[1 - 1, :])

0    3.0
1    4.0
2    5.0
3    6.0
4    7.0
5    8.0
6    9.0
dtype: float64

In [50]:
np.array([[3.], [4.], [5.], [6.], [7.], [8.], [9.]])

array([[3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]])

In [53]:
np.array([[6., 7., 8.],
                  [7., 8., 9.]]).transpose()

array([[6., 7.],
       [7., 8.],
       [8., 9.]])